# QuickStart
Based on [Pytorch Quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

- This section runs through the API for common tasks in ML.

## Working with data

PyTorch has 2 primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`.

`Dataset` stores the samples and their corresponding labels, and `Dataloader` wraps an iterable around the `Dataset`.

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch has TorchText, TorchVision and TorchAudio, all contain datasets.

We will use a TorchVision dataset.

The `torchvision.datasets` module contains `Dataset` objects for many real-world vision data like CIFAR, COCO etc

Here is the [full list](https://pytorch.org/vision/stable/datasets.html)

We'll use a 3 datasets to get the gist: FashionMNIST.

Every TorchVision Dataset includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

In [14]:
# download training data from open datasets

train_data = datasets.FashionMNIST(
root = 'data',
download=True
) #required argument = `root` if locally available else #2nd required argument is download=True 

In [15]:
dummy_train_data = datasets.CelebA(root='data1',download=True)

2422it [00:00, 19167177.90it/s]
/opt/conda/lib/python3.10/site-packages/torchvision/datasets/utils.py:260: UserWarning: We detected some HTML elements in the downloaded file. This most likely means that the download triggered an unhandled API response by GDrive. Please report this to torchvision at https://github.com/pytorch/vision/issues including the response:

<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv="content-type" content="text/html; charset=utf-8"/><style nonce="83cdZ0QgQKzixtOiBuH7yg">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial,sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.

RuntimeError: The MD5 checksum of the download file data1/celeba/img_align_celeba.zip does not match the one on record.Please delete the file and try again. If the issue persists, please report this to torchvision at https://github.com/pytorch/vision/issues.